In [112]:
import numpy as np

class KNearestNeighbors():
    def __init__(self, X, y, n_neighbors=5, weights = 'uniform', p=2, metric = 'minkowski'):
        self.X = X
        self.y = y
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.p = p
        self.metric = metric
        self.class_num = 3
    def distance(self, x, y):
      #  if self.p==2 & self.weights =='uniform':
            
        return np.sqrt(np.sum((x-y)**2, axis=1))
    
    def voting():
        pass
    
    
    def kneighbors(self, X_test, return_distance=False):
       
        #neigh_ind = np.array([np.argsort(self.distance(x_test, self.X))[:self.n_neighbors] for x_test in X_test])
        #dist = np.array([np.sort(self.distance(x_test, X))[:self.n_neighbors] for x_test in X_test])
        
        dist = []
        neigh_ind = []
        
        point_dist = [self.distance(x_test, self.X) for x_test in X_test]

        for row in point_dist:
            enum_neigh = enumerate(row)
            sorted_neigh = sorted(enum_neigh, key=lambda x: x[1])[:self.n_neighbors]
    
            ind_list = [tup[0] for tup in sorted_neigh]
            dist_list = [tup[1] for tup in sorted_neigh]
    
            dist.append(dist_list)
            neigh_ind.append(ind_list)
        
        if return_distance:
            return np.array(dist), np.array(neigh_ind)
        
        return np.array(neigh_ind)
         
    
    def predict(self, X_test):
        
        if self.weights=='uniform':
            neighbors = self.kneighbors(X_test)
            y_pred = np.array([np.argmax(np.bincount(y[neighbor])) for neighbor in neighbors])
        
            return y_pred
    
        if self.weights=='distance':
        
            dist, neigh_ind = self.kneighbors(X_test, return_distance=True)
        
            inv_dist = 1/dist
            
            mean_inv_dist = inv_dist / np.sum(inv_dist, axis=1)[:, np.newaxis]
            
            proba = []
            
            for i, row in enumerate(mean_inv_dist):
                
                row_pred = y[neigh_ind[i]]
                
                for k in range(self.class_num):
                    indices = np.where(row_pred==k)
                    prob_ind = np.sum(row[indices])
                    proba.append(np.array(prob_ind))
        
            predict_proba = np.array(proba).reshape(X_test.shape[0], self.class_num)
            
            y_pred = [np.argmax(item) for item in predict_proba]
            
            return y_pred
            
    def score(self, X, y):
        pass


In [113]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples = 1000, n_features=2, n_redundant=0, n_informative=2,
                             n_clusters_per_class=1, n_classes=3, random_state=21)


mu = np.mean(X, 0)
sigma = np.std(X, 0)

X = (X - mu ) / sigma

In [122]:
knn = KNearestNeighbors(X, y, weights='uniform')

In [123]:
t = np.array([[1,2], [33, 88], [23, 2]])

In [124]:
knn.kneighbors(t)

array([[708,  98, 493, 958, 430],
       [ 31, 286, 841, 708, 204],
       [101, 531, 730, 320, 838]])

In [125]:
n = knn.predict(np.array([[1,2],[2,3], [-0.7,-1.1], [444, 91], [14, -5],[-2,-3] ]))

In [126]:
n.shape

(6,)

In [127]:
n

array([0, 0, 2, 0, 0, 2])

In [121]:
[np.argmax(i) for i in n]

[0, 0, 2, 0, 0, 2]

In [37]:
neighbor = knn.kneighbors(np.array([[1,2],[1,5], [-0.7,-1.1]]))

In [38]:
neighbor

[2, 0, 1]